In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# **加载数据**
X_train = np.load("D:\Dataset\Temp_Results\Wavelet\X_train_cwt.npy")  # 形状: (num_samples, 128, 128)
Y_train = np.load("D:\Dataset\Marine-Mammal-Target-Classification\01_PreProcess\y_train.npy")  # 形状: (num_samples,)
X_test = np.load("D:\Dataset\Temp_Results\Wavelet\X_test_cwt.npy")  # 形状: (num_samples, 128, 128)
Y_test = np.load("D:\Dataset\Marine-Mammal-Target-Classification\01_PreProcess\y_test.npy")  # 形状: (num_samples,)

# **数据预处理**
X_train = X_train.astype(np.float32) / 255.0  # 归一化到 [0,1]
X_train = np.expand_dims(X_train, axis=-1)  # 添加通道维度 (num_samples, 128, 128, 1)

# **转换标签格式**
num_classes = len(np.unique(Y_train))  # 获取类别数量
Y_train = to_categorical(Y_train, num_classes)  # 转换为 One-Hot 编码

# **划分训练集和测试集**
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# **构建 CNN 模型**
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # 随机丢弃 50% 神经元，防止过拟合
    Dense(num_classes, activation='softmax')  # 多分类: softmax, 二分类: sigmoid
])

# **编译模型**
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# **训练模型**
history = model.fit(
    X_train, Y_train, 
    epochs=20, batch_size=32, 
    validation_data=(X_val, Y_val)
)

# **保存模型**
model.save("cnn_wavelet_model.h5")

print("✅ CNN 训练完成，模型已保存！")
